In [1]:
import os
import sys
import importlib
import json
import numpy as np
import pandas as pd
import scipy.ndimage as snd
import skimage
import uuid

if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir("../..")
directory_path = os.path.abspath(os.path.join("src"))
if directory_path not in sys.path:
    sys.path.append(directory_path)

import EyeTraumaAnalysis

print(directory_path)
importlib.reload(EyeTraumaAnalysis);
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import cv2

import plotly.express as px
import plotly


/Users/ryerrabelli/Library/CloudStorage/GoogleDrive-ryerrabelli@gmail.com/My Drive/Computer Backups/Rahul Yerrabelli drive/PythonProjects/EyeTraumaAnalysis/src


# Load data from excel

In [2]:
kmeans_labels = pd.read_excel("data/01_raw/Ergonautus/Ergonautus_Clusters_Correct_Values.xlsx", dtype={
    "Correct 1" : "Int64",  # "Int64" is from pandas, unlike int64 and allows null
    "Correct 2" : "Int64",
    "Correct 3" : "Int64",
    "Borderline": "Int64",
    "Notes"     : str,
    "Filename"  : str,
}, na_filter=False)  # False na_filters make empty value for str column be "" instead of NaN

# Calculate metrics

In [3]:
all_metrics = []
all_kmeans_masks = {}
for ind, filename in enumerate(kmeans_labels["Filename"]):
    img_bgr = skimage.io.imread(os.path.join("data/01_raw/", filename))
    centers, ranges, res_bgr, kmeans_masks = EyeTraumaAnalysis.kmeans.create_kmeans(img_bgr)
    metrics = EyeTraumaAnalysis.kmeans.get_kmeans_metrics(centers, ranges, kmeans_masks)
    all_metrics.append(metrics)
    all_kmeans_masks[filename] = kmeans_masks

all_metrics = pd.concat(all_metrics, keys=kmeans_labels["Filename"])

In [4]:
all_metrics.loc[:, ("Labels", "Value", "", "")] = "False"
all_metrics.loc[:, ("Labels", "Correct", "", "")] = False
all_metrics.loc[:, ("Labels", "Borderline", "", "")] = False
for ind, row in kmeans_labels.iterrows():
    filename = row["Filename"]

    correct_indices = row[["Correct 1", "Correct 2", "Correct 3"]] - 1  # subtract by 1 to go from 1-10 to 0-9
    correct_indices = correct_indices[~pd.isnull(correct_indices)].to_numpy().astype(int)
    file_maskrow_correct_indices = [(filename, correct_index) for correct_index in correct_indices]
    all_metrics.loc[file_maskrow_correct_indices, ("Labels", "Correct", "", "")] = True
    all_metrics.loc[file_maskrow_correct_indices, ("Labels", "Value", "", "")] = "True"

    borderline_indices = row[["Borderline"]] - 1
    borderline_indices = borderline_indices[~pd.isnull(borderline_indices)].to_numpy().astype(int)
    file_maskrow_borderline_indices = [(filename, borderline_index) for borderline_index in borderline_indices]
    all_metrics.loc[file_maskrow_borderline_indices, ("Labels", "Borderline", "", "")] = True
    all_metrics.loc[file_maskrow_borderline_indices, ("Labels", "Value", "", "")] = "Maybe"

# Reorder
all_metrics = all_metrics[["Labels", "Ranks", "Values"]]
# Label the second index
all_metrics.index.names = [all_metrics.index.names[0], "Mask"]

In [5]:
all_metrics_agg = all_metrics.groupby([("Labels", "Value")]).agg(["median"])[["Ranks", "Values"]]


/Users/ryerrabelli/opt/anaconda3/envs/EyeTraumaAnalysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3135: PerformanceWarning: indexing past lexsort depth may impact performance.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/ryerrabelli/opt/anaconda3/envs/EyeTraumaAnalysis/lib/python3.10/site-packages/pandas/core/groupby/generic.py:979: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = op.agg()


# Prepare for plot / Create flat version of df

In [6]:
all_metrics_flat = all_metrics.copy()
all_metrics_flat.columns = ["-".join(multi_col).rstrip("-") for multi_col in all_metrics.columns]
all_metrics_flat = all_metrics_flat.reset_index()
all_metrics_flat["Ranks-Color-Center-V>4"] = all_metrics_flat["Ranks-Color-Center-V"] > 4
all_metrics_flat["Ranks-Color-Center-V>5"] = all_metrics_flat["Ranks-Color-Center-V"] > 5

In [7]:
import plotly.express as px

var_labels = {
    "Labels-Value"          : "Conjunctiva cluster",
    "Values-Color-Center-H" : "Center H",
    "Values-Color-Center-S" : "Center S",
    "Values-Color-Center-V" : "Center V",
    "Values-Color-Range-H"  : "Range H",
    "Values-Color-Range-S"  : "Range S",
    "Values-Color-Range-V"  : "Range V",
    "Values-Location-Mean-x": "Mean x",
    "Values-Location-Mean-y": "Mean y",
    "Values-Location-SD-x"  : "SD x",
    "Values-Location-SD-y"  : "SD y",
    "Ranks-Color-Mean-V>4"  : "V Rank >4",
    "Ranks-Color-Mean-V>5"  : "V Rank >5"
}
color_discrete_map = {
    "True" : px.colors.qualitative.Safe[0],  # green
    "Maybe": px.colors.qualitative.Safe[2],  # blue
    "False": px.colors.qualitative.Safe[1],  # red
}
category_orders = {
    "Labels-Value": ["False", "Maybe", "True"],
}
for var_label_key in var_labels.copy():
    if var_label_key.startswith("Values-"):
        var_label_key_suffix = var_label_key.split("Values-", maxsplit=1)[-1]
        var_labels[f"Ranks-{var_label_key_suffix}"] = var_labels[var_label_key] + " (Rank)"
plotly_template = "plotly_dark"  #"simple_white"


def save_plotly_figure(fig, title):
    fig.write_image("outputs/kmeans-descriptive/" + title + ".png")
    fig.write_html("outputs/kmeans-descriptive/" + title + ".html",
                   full_html=True,
                   include_plotlyjs="directory")

In [9]:
all_metrics_flat[all_metrics_flat["Labels-Value"]=="True"]

Labels                     Ranks                                \
        Value Correct Borderline  Color                                 
                                 Center             Range               
                                      H     S     V     H     S     V   
min     False   False      False    1.0   1.0   1.0   1.0   1.0   1.0   
median    NaN     0.0        0.0    5.5   5.5   5.5   5.5   5.5   5.5   
max      True    True       True   10.0  10.0  10.0  10.0  10.0  10.0   

                 ... Values                                                  \
       Location  ...  Color                              Location             
           Mean  ... Center         Range                    Mean             
              x  ...      S      V      H      S      V         x         y   
min         1.0  ...    0.0    1.0    0.0    0.0    0.0  0.191721  0.126726   
median      5.5  ...   92.0  112.0   75.5  120.5   94.0  0.491321  0.396362   
max        10.0  ...  255.0  253.0  255.0  254.0  249.0  0.797200  0.857627   

                                      
                                Area  
              SD                      
               x         y            
min     0.000000  0.000000  0.000045  
median  0.168796  0.156880  0.062062  
max     0.381592  0.408404  0.478826  

[3 rows x 25 columns]

In [12]:
all_metrics.groupby([("Labels","Value")]).agg(["min","median","max"]).to_excel(
    "outputs/kmeans-descriptive/aggregate.xlsx")

/Users/ryerrabelli/opt/anaconda3/envs/EyeTraumaAnalysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3135: PerformanceWarning: indexing past lexsort depth may impact performance.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/ryerrabelli/opt/anaconda3/envs/EyeTraumaAnalysis/lib/python3.10/site-packages/pandas/core/groupby/generic.py:979: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = op.agg()
